### 이전 코드 세팅

In [2]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional

class GoogleNews:
    """
    구글 뉴스를 검색하고 결과를 반환하는 클래스입니다.
    """

    def __init__(self):
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:

        if not news_list:
            print("해당 키워드의 뉴스가 없습니다.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=ko&gl=KR&ceid=KR:ko"
        else:
            url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

In [3]:
from dotenv import load_dotenv
from typing import Annotated, List, Dict
from typing_extensions import TypedDict

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()

class State(TypedDict):
    messages: Annotated[list, add_messages]

news_tool = GoogleNews()


@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


tools = [search_keyword]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)

graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [4]:
from langchain_core.runnables import RunnableConfig

# 질문
question = "LangGraph 가 무엇인지 조사하여 알려주세요!"

# 초기 입력 상태를 정의
input = State(messages=[("user", question)])

# config 설정
config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [5]:
list(graph.channels)

['messages',
 '__start__',
 'chatbot',
 'tools',
 'start:chatbot',
 'branch:chatbot:tools_condition:chatbot',
 'branch:chatbot:tools_condition:tools']

tools 에서 interrupt before 걸리도록

In [6]:
# 그래프 스트림 호출
events = graph.stream(
    input=input, config=config, interrupt_before=["tools"], stream_mode="values"
)

# 이벤트 반복 처리
for event in events:
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지의 예쁜 출력
        event["messages"][-1].pretty_print()

================================ Human Message =================================

LangGraph 가 무엇인지 조사하여 알려주세요!
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_zyzjRbv24s72LH31uFs7eUXR)
 Call ID: call_zyzjRbv24s72LH31uFs7eUXR
  Args:
    query: LangGraph


In [7]:
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)

# 가장 최근 메시지 추출
last_message = snapshot.values["messages"][-1]

# 메시지 출력
last_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_zyzjRbv24s72LH31uFs7eUXR)
 Call ID: call_zyzjRbv24s72LH31uFs7eUXR
  Args:
    query: LangGraph


### 지금부터 진행하는 것은 사람이 직접 개입해서 상태를 바꾸는 것이다.

#### 예시로 진행하는 것은 tool 호출 부분을 개입해서 바꿀 것이다.

먼저 도구 호출 id 획득

In [8]:
tool_call_id = last_message.tool_calls[0]["id"]
print(tool_call_id)

call_zyzjRbv24s72LH31uFs7eUXR


In [9]:
modified_search_result = """[수정된 웹 검색 결과] 
LangGraph 말고 LangChain으로 검색하세요.

테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요."""

print(modified_search_result)

[수정된 웹 검색 결과] 
LangGraph 말고 LangChain으로 검색하세요.

테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.


상태 변경

In [10]:
from langchain_core.messages import AIMessage, ToolMessage

new_messages = [
    # LLM API의 도구 호출과 일치하는 ToolMessage 필요
    ToolMessage(
        content=modified_search_result,
        tool_call_id=tool_call_id,
    ),
    # LLM의 응답에 직접적으로 내용 추가
    # AIMessage(content=modified_search_result),
]

new_messages[-1].pretty_print()

================================= Tool Message =================================

[수정된 웹 검색 결과] 
LangGraph 말고 LangChain으로 검색하세요.

테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.


다시 시작할 때 변경된 메시지를 집어 넣을 예정

In [11]:
graph.update_state(
    # 업데이트할 상태 지정
    config,
    # 제공할 업데이트된 값. `State`의 메시지는 "추가 전용"으로 기존 상태에 추가됨
    {"messages": new_messages},
    as_node="tools",
)

print("(최근 1개의 메시지 출력)\n")
print(graph.get_state(config).values["messages"][-1])

(최근 1개의 메시지 출력)

content='[수정된 웹 검색 결과] \nLangGraph 말고 LangChain으로 검색하세요.\n\n테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.' id='7cfe0022-57f1-4612-af67-8e8a058bb7dd' tool_call_id='call_zyzjRbv24s72LH31uFs7eUXR'


In [12]:
# `None`는 현재 상태에 아무것도 추가하지 않음
events = graph.stream(None, config, stream_mode="values")

# 이벤트 반복 처리
for event in events:
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지의 예쁜 출력
        event["messages"][-1].pretty_print()

================================= Tool Message =================================

[수정된 웹 검색 결과] 
LangGraph 말고 LangChain으로 검색하세요.

테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_iiRF0fGqOoCEQiq6dJdgRcts)
 Call ID: call_iiRF0fGqOoCEQiq6dJdgRcts
  Args:
    query: LangChain
================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMieEFVX3lxTE15VWVUMHAxYVdMcmpKWnhHLXVRZnh4LVB5cnd0Vldia2ZlRmNGMnByWVVlWXJMakhyZTJKVlJvMWRIRkRjVTFuUE52WU9oVnN2VkM1UTRnWThFLVRaaE04TDZKM0VzbmNJMkFLLWoyNzc0SktDVXJ4Rw?oc=5", "content": "가볍게 훑어보는 LangChain (랭체인) | 인사이트리포트 | 삼성SDS - Samsung SDS"}, {"url": "https://news.google.com/rss/articles/CBMisANBVV95cUxOLXNzMUpYX3NHa1FVMkI0TTFmSGRFZ05vVTNKYnlta04wRUF0SlRVLXltbHhoQk1BcjJHR1ZFRFJLZnFUMGxveXl0MElNdXBVZjRQeDVrZkdRczU5MEY2RFZEb1ZZU21fRE44X0ZtUko0Qm

In [15]:
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)

# 최근 세 개의 메시지 출력
for message in snapshot.values["messages"]:
    message.pretty_print()

================================ Human Message =================================

LangGraph 가 무엇인지 조사하여 알려주세요!
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_zyzjRbv24s72LH31uFs7eUXR)
 Call ID: call_zyzjRbv24s72LH31uFs7eUXR
  Args:
    query: LangGraph
================================= Tool Message =================================

[수정된 웹 검색 결과] 
LangGraph 말고 LangChain으로 검색하세요.

테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_iiRF0fGqOoCEQiq6dJdgRcts)
 Call ID: call_iiRF0fGqOoCEQiq6dJdgRcts
  Args:
    query: LangChain
================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMieEFVX3lxTE15VWVUMHAxYVdMcmpKWnhHLXVRZnh4LVB5cnd0Vldia2ZlRmNGMnByWVVlWXJMakhyZTJKVlJvMWRIRkRjVTFuUE52WU9oVnN2VkM1UTRnWThFLVRa

### 예시 한 개 더 추가

In [16]:
import random

def generate_random_hash():
    return f"{random.randint(0, 0xffffff):06x}"

thread_id = generate_random_hash()
print(f"thread_id: {thread_id}")

question = "LangGraph 에 대해서 배워보고 싶습니다. 유용한 자료를 추천해 주세요!"

# 초기 입력 상태를 정의
input = State(messages=[("user", question)])

# 새로운 config 생성
config = {"configurable": {"thread_id": thread_id}}

events = graph.stream(
    input=input,
    config=config,
    interrupt_before=["tools"],
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

thread_id: 791024
================================ Human Message =================================

LangGraph 에 대해서 배워보고 싶습니다. 유용한 자료를 추천해 주세요!
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_lEVcNbZeUEGDs6tnFOVTTY3Q)
 Call ID: call_lEVcNbZeUEGDs6tnFOVTTY3Q
  Args:
    query: LangGraph


In [17]:
config_copy = config.copy()

In [18]:
from langchain_core.messages import AIMessage

# 스냅샷 상태 가져오기
snapshot = graph.get_state(config)

# messages 의 마지막 메시지 가져오기
existing_message = snapshot.values["messages"][-1]

# 메시지 ID 출력
print("Message ID", existing_message.id)

Message ID run-6f9b8ef9-72d8-4d24-b4ee-485e596cc632-0


In [19]:
# tool_calls 를 복사하여 새로운 도구 호출 생성
new_tool_call = existing_message.tool_calls[0].copy()

# 쿼리 매개변수 업데이트(갱신)
new_tool_call["args"] = {"query": "LangGraph site:teddylee777.github.io"}
new_tool_call

{'name': 'search_keyword',
 'args': {'query': 'LangGraph site:teddylee777.github.io'},
 'id': 'call_lEVcNbZeUEGDs6tnFOVTTY3Q',
 'type': 'tool_call'}

Update

In [20]:
# AIMessage 생성
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[new_tool_call],
    # 중요! ID는 메시지를 상태에 추가하는 대신 교체하는 방법
    id=existing_message.id,
)

print(new_message.id)

# 수정한 메시지 출력
new_message.pretty_print()

run-6f9b8ef9-72d8-4d24-b4ee-485e596cc632-0
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_lEVcNbZeUEGDs6tnFOVTTY3Q)
 Call ID: call_lEVcNbZeUEGDs6tnFOVTTY3Q
  Args:
    query: LangGraph site:teddylee777.github.io


In [21]:
# 상태 업데이트
graph.update_state(config, {"messages": [new_message]})

{'configurable': {'thread_id': '791024',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef9beec-31dc-65e8-8002-72e40468b90b'}}

In [22]:
events = graph.stream(None, config, stream_mode="values")

# 각 이벤트에 대한 처리
for event in events:
    # 메시지가 있는 경우 마지막 메시지 출력
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_lEVcNbZeUEGDs6tnFOVTTY3Q)
 Call ID: call_lEVcNbZeUEGDs6tnFOVTTY3Q
  Args:
    query: LangGraph site:teddylee777.github.io
================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMiREFVX3lxTE9YdjI3SVhReEpSc2ctaUl1ZzBpb2hFbUxpRnFacGtCRHdROEoyNjlVVkZJS09RdnlqSml6U2d0RGk4a1JG?oc=5", "content": "테디노트 - GitHub Pages"}, {"url": "https://news.google.com/rss/articles/CBMib0FVX3lxTE9iTXZZTlNoMEw1QnVvXzJZUTRyT19CanNUbUZicUVqNGtIdWFTSnprb2VoYU9oRUZGVDNRTVhsa2RIT2xBVnVVUXY4SXJtWXpJVW1fZ2xUQ1g2VWZqQnFyMklXc3FiYzJDb056eTM2NA?oc=5", "content": "LangGraph Retrieval Agent를 활용한 동적 문서 검색 및 처리 - GitHub Pages"}, {"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE9BVEhGRGZGLXduVDd5LVJKSk1TSUwwNHdGb0xWVjZtU2FIdF9GZjZVbDRhaWRFOGpoVEc3WHlubFFWVGs3UnVKT18yTlRwcjZEaGFkaGtXdjJWei13X3o4NA?oc=5", "c

In [23]:
# 이벤트 스트림 생성
events = graph.stream(
    {
        "messages": (
            "user",
            "내가 지금까지 배운 내용에 대해서 매우 친절하고 정성스럽게 한국어로 답변해줘! 출처를 반드시 포함해줘!",
        )
    },
    config,
    stream_mode="values",
)

# 메시지 이벤트 처리
for event in events:
    if "messages" in event:
        # 마지막 메시지 출력
        event["messages"][-1].pretty_print()

================================ Human Message =================================

내가 지금까지 배운 내용에 대해서 매우 친절하고 정성스럽게 한국어로 답변해줘! 출처를 반드시 포함해줘!
================================== Ai Message ==================================

LangGraph에 대해 배운 내용을 정리해 드리겠습니다.

1. **LangGraph Retrieval Agent**: 이 도구는 동적 문서 검색 및 처리에 사용됩니다. 사용자는 LangGraph를 활용하여 특정한 정보나 데이터를 효율적으로 검색하고, 이를 기반으로 다양한 작업을 수행할 수 있습니다. 이 자료는 LangGraph의 기능과 활용 방법을 자세히 설명하고 있습니다. [자세한 내용 보기](https://news.google.com/rss/articles/CBMib0FVX3lxTE9iTXZZTlNoMEw1QnVvXzJZUTRyT19CanNUbUZicUVqNGtIdWFTSnprb2VoYU9oRUZGVDNRTVhsa2RIT2xBVnVVUXY4SXJtWXpJVW1fZ2xUQ1g2VWZqQnFyMklXc3FiYzJDb056eTM2NA?oc=5)

2. **문서 기반 QA 시스템 설계**: LangChain과 LangGraph를 활용하여 문서 기반의 질문-답변(QA) 시스템을 설계하는 방법에 대한 심화 자료입니다. 이 자료에서는 시스템의 구조와 기능, 그리고 구현 방법에 대해 깊이 있는 내용을 다루고 있습니다. [자세한 내용 보기](https://news.google.com/rss/articles/CBMiY0FVX3lxTE13clZKZXp0SVZ6RmJORVE1SnVtZUtGVk9mNlg0bzY4Y191N3ozWEFwUzF5TkJRTkdOMlVvZEljWHNJeTB0UFM3d1JMajZWTkthQlJmVjJTRG8zTTI4VkZ5dnAxSQ?oc=5)

이 두 가지 자료

### replay에 update 적용

먼저 replay 할 곳 지정

In [24]:
to_replay_state = None

# 상태 기록 가져오기
for state in graph.get_state_history(config):

    messages = state.values["messages"]

    if len(messages) > 0:
        print(state.values["messages"][-1].id)
        # 메시지 수 및 다음 상태 출력
        print("메시지 수: ", len(state.values["messages"]), "다음 노드: ", state.next)
        print("-" * 80)
        # 특정 상태 선택 기준: 채팅 메시지 수
        if len(state.values["messages"]) == 2:
            # 특정 메시지 ID 선택
            to_replay_state = state

run-9b5802f7-0021-4b45-90cd-2cfc2292d5ea-0
메시지 수:  6 다음 노드:  ()
--------------------------------------------------------------------------------
220db059-14e2-4abe-b429-afdfea20e05b
메시지 수:  5 다음 노드:  ('chatbot',)
--------------------------------------------------------------------------------
run-886231d4-594a-4bbd-b613-74f0bfbb7a7d-0
메시지 수:  4 다음 노드:  ('__start__',)
--------------------------------------------------------------------------------
run-886231d4-594a-4bbd-b613-74f0bfbb7a7d-0
메시지 수:  4 다음 노드:  ()
--------------------------------------------------------------------------------
295f8c2a-0964-4497-bc79-efa6f149893d
메시지 수:  3 다음 노드:  ('chatbot',)
--------------------------------------------------------------------------------
run-6f9b8ef9-72d8-4d24-b4ee-485e596cc632-0
메시지 수:  2 다음 노드:  ('tools',)
--------------------------------------------------------------------------------
run-6f9b8ef9-72d8-4d24-b4ee-485e596cc632-0
메시지 수:  2 다음 노드:  ('tools',)
------------------------------

In [25]:
# 선택한 메시지 가져오기
existing_message = to_replay_state.values["messages"][-1]

메시지 수정

In [29]:
tool_call = existing_message.tool_calls[0].copy()
tool_call["args"] = {"query": "미국 대선 관련 뉴스"}
# AIMessage 생성
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[tool_call],
    # 중요! ID는 메시지를 상태에 추가하는 대신 교체하는 방법
    id=existing_message.id,
)

# 상태 업데이트
updated_state = graph.update_state(
    to_replay_state.config,
    {"messages": [new_message]},
)

업데이트 된 질문 스트림

In [30]:
# config 에는 updated_state 를 전달합니다. 이는 임의로 갱신한 상태를 전달하는 것입니다.
for event in graph.stream(None, updated_state, stream_mode="values"):
    # 메시지가 이벤트에 포함된 경우
    if "messages" in event:
        # 마지막 메시지 출력
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_lEVcNbZeUEGDs6tnFOVTTY3Q)
 Call ID: call_lEVcNbZeUEGDs6tnFOVTTY3Q
  Args:
    query: 미국 대선 관련 뉴스
================================= Tool Message =================================
Name: search_keyword

[{"url": "https://news.google.com/rss/articles/CBMiXEFVX3lxTE5ZeGtTRGowektQaktCUXdJbjJMN1pWS05UUVpXckpwWUFhcTJpNWgtOV9tMHBCRGRFVmc0NERVZVNXcUFUZlF4VWhzbGZCeXlEanhncGlDRVlrbnVX0gFiQVVfeXFMTWtuQUtrSXVEdkNZay1WZ0FrOVFMT1NGSXBXY2dHTDQ1ZklMNHNqQlNTZHFBbnVuVURKa2Q2OHg5cW9PcGRVajBYV2FpV0lhYjl2WU1rWHZDdGI4V2p5VzhNTXc?oc=5", "content": "2024 미국 대선 개표 현황 - BBC.com"}, {"url": "https://news.google.com/rss/articles/CBMiUEFVX3lxTFBUT2NiQ0NMWEI5a2NjTE9yYzJ4UG81and1WE1JWEVDWHI4bXlXLTNwenVnU2puODVtZy0yRGhvYktDSmE5ZlB6TUlqaU90RkIy?oc=5", "content": "2024 미국 대선 - 동아일보"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE9GblJYOG9CNTRXU0dqVTc4WEwteC1obTJhRm9ZNC1rczJGSXhPRzVfb3oxQ1dTOVc